# 上分思路
- 数据
- 模型
- 损失函数
- 训练方式
- 超参数

### **第一步**
在运行环境中安装对应的库 执行该命令即可

In [ ]:
! pip install -r requirements.txt

### **第二步**
导入运行所需要的库函数 

In [4]:
import os
import pandas as pd
import xarray as xr
from torch.utils.data import Dataset, DataLoader

### **第三步**
数据集路径配置设置
- 比赛的数据部分分为**数据特征**和**数据真值**两部分，数据特征是模型训练的**输入**，数据真值是模型训练的**标签**
- 其中数据特征部分 输入的路径目录下包含年份文件夹 
 - 例如示例给出的 "输入路径/2021/..." 各年份文件夹下包含从官网下载的压缩包(e.g. weather.round1.train.ft.2021.1.zip) 解压后文件夹下有不同时段的数据文件夹(e.g. 20210101-00), 内部包含6个nc文件, 是从伏羲大模型中获取的从第6小时到第72小时的数据

- 数据真值部分 输入的路径目录下包含3个年份的.nc数据, 其中选择哪些年份的特征数据作为输入, 就在years中添加哪些年份
- fcst_steps指预测的时间步长, 从第1小时到第72小时, 间隔为1小时



In [5]:
# path config
feature_path = '/ai_share/caozhibin/tianchi_precipatition_prediction/train' #自定义路径并修改为自己的路径
gt_path = '/ai_share/caozhibin/tianchi_precipatition_prediction/groundtruth' #自定义路径并修改为自己的路径
years = ['2021']
fcst_steps = list(range(1, 73, 1))

### **第四步**
Feature类和GroundTruth类是数据集的定义
方便后续自定义数据集和数据加载类, 方便我们训练时取数据

In [6]:
# Feature部分
class Feature:
    def __init__(self):
        self.path = feature_path
        self.years = years
        self.fcst_steps = fcst_steps
        self.features_paths_dict = self.get_features_paths()

    def get_features_paths(self):
        init_time_path_dict = {}
        for year in self.years:
            init_time_dir_year = os.listdir(os.path.join(self.path, year))
            for init_time in sorted(init_time_dir_year):
                init_time_path_dict[pd.to_datetime(init_time)] = os.path.join(self.path, year, init_time)
        return init_time_path_dict

    def get_fts(self, init_time):
        return xr.open_mfdataset(self.features_paths_dict.get(init_time) + '/*').sel(lead_time=self.fcst_steps).isel(
            time=0)
    
# GroundTruth部分
class GT:
    def __init__(self):
        self.path = gt_path
        self.years = years
        self.fcst_steps = fcst_steps
        self.gt_paths = [os.path.join(self.path, f'{year}.nc') for year in self.years]
        self.gts = xr.open_mfdataset(self.gt_paths)

    def parser_gt_timestamps(self, init_time):
        return [init_time + pd.Timedelta(f'{fcst_step}h') for fcst_step in self.fcst_steps]

    def get_gts(self, init_time):

        return self.gts.sel(time=self.parser_gt_timestamps(init_time))

### **第五步**
mydataset类的定义, 整合了加载特征和特征对应真值的功能, 方便后续训练时取数据

In [7]:
# 构建Dataset部分
class mydataset(Dataset):
    def __init__(self):
        self.ft = Feature()
        self.gt = GT()
        self.features_paths_dict = self.ft.features_paths_dict
        self.init_times = list(self.features_paths_dict.keys())

    def __getitem__(self, index):
        init_time = self.init_times[index]
        try:
            ft_item = self.ft.get_fts(init_time).to_array().isel(variable=0).values
            gt_item = self.gt.get_gts(init_time).to_array().isel(variable=0).values
        except KeyError as e:
            print(e)
            print(f'init_time: {init_time} not found')
            # return None, None
            return self.__getitem__(index - 1)
        
        return ft_item, gt_item

    def __len__(self):
        return len(list(self.init_times))

### **第六步**
前五步已经完成了数据预处理加载的相关类和函数的准备, 这里我们可以通过实例化mydataset类来查看数据数量
同时完成数据集的构建后, 我们可以通过DataLoader来查看数据集的数据

In [8]:
import torch
# define dataset
my_data = mydataset()
print('sample num:', mydataset().__len__())

sample num: 724


In [9]:
from torch.utils.data import Dataset, DataLoader, random_split
# Split the dataset into training and validation sets
train_size = int(0.8 * len(my_data))
val_size = len(my_data) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(my_data, [train_size, val_size])

# Create data loaders for training and validation sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

### **第七步**
- 完成了数据的准备工作, 接下来就是构建模型的部分
- Model这个类, 对我们的模型进行定义, 方便后续训练时调用
- 这里我们以一个简单的只有一个卷积层的网络为例
- 在本次比赛中, 我们的输入数据维度是(1, 24, 72, W, H), 输出数据维度是(1, 72, W, H) 可以在赛题中查看

In [30]:
import torch
import torch.nn as nn

# 实验1 加深模型
class EnhancedModel(nn.Module):
    def __init__(self, num_in_ch, num_out_ch):
        super(EnhancedModel, self).__init__()
        self.conv1 = nn.Conv2d(num_in_ch, 64, kernel_size=3, padding=1)
        self.batchnorm = nn.BatchNorm2d(64)
        self.activation = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, num_out_ch, kernel_size=3, padding=1)
    
    def forward(self, x):
        B, S, C, W, H = tuple(x.shape)
        x = x.reshape(B, -1, W, H)
        out = self.conv1(x)
        out = self.activation(out)
        out = self.conv2(out)
        out = self.activation(out)
        out = self.conv3(out)
        out = self.activation(out)
        out = self.conv4(out)
        out = self.activation(out)
        out = out.reshape(B, S, W, H)
        return out
    
# define model
in_varibales = 24
in_times = len(fcst_steps)
out_varibales = 1
out_times = len(fcst_steps)
input_size = in_times * in_varibales
output_size = out_times * out_varibales
model = EnhancedModel(input_size, output_size).cuda()

In [20]:
# 推荐先使用这个网络
import torch
import torch.nn as nn

class EnhancedModel(nn.Module):
    def __init__(self, num_in_ch, num_out_ch):
        super(EnhancedModel, self).__init__()
        self.conv = nn.Conv2d(num_in_ch, num_out_ch, kernel_size=3, padding=1)
        self.activation = nn.ReLU()


    def forward(self, x):
        B, S, C, W, H = tuple(x.shape)
        x = x.reshape(B, -1, W, H)
        out = self.conv(x)
        out = self.activation(out)
        out = out.reshape(B, S, W, H)
        return out

# define model
in_varibales = 24
in_times = len(fcst_steps)
out_varibales = 1
out_times = len(fcst_steps)
input_size = in_times * in_varibales
output_size = out_times * out_varibales
model = EnhancedModel(input_size, output_size).cuda()

### **第八步**
定义模型的损失函数部分， 用于模型训练做反向传播

In [31]:
loss_func = nn.SmoothL1Loss()
# loss_func = nn.MSELoss()

### **第九步**
模型训练部分

In [32]:
# 模型初始化
import torch.nn.init as init
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.constant_(m.bias, 0)

model.apply(init_weights)

EnhancedModel(
  (conv1): Conv2d(1728, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batchnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 72, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [34]:
import numpy as np
import torch
# from tqdm import tqdm
# Train the model
num_epochs = 20
optimizer = torch.optim.Adam(model.parameters(), lr=0.0004, weight_decay=1e-6)

# for epoch in tqdm(range(num_epochs)):
os.makedirs('./model', exist_ok=True)
for epoch in range(num_epochs):
    model.train()
    loss = 0.0
    for index, (ft_item, gt_item) in enumerate(train_loader):
        ft_item = ft_item.cuda().float()
        gt_item = gt_item.cuda().float()
        # print("gt", gt_item.max(), gt_item.min())
        # Backward and optimize
        optimizer.zero_grad()
        # Forward pass
        output_item = model(ft_item)
        # print(output_item.max(), output_item.min())
        loss = loss_func(output_item, gt_item)
            
        loss.backward()
        optimizer.step()
            
        loss += loss.item()
        # Print the loss for every 10 steps
        if (index+1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{index+1}/{len(train_loader)}], Loss: {loss.item():.6f}")
            loss = 0.0
    # Save the model weights
    torch.save(model.state_dict(), f'./model/model_weights_{epoch}.pth')
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for index, (ft_item, gt_item) in enumerate(val_loader):
            ft_item = ft_item.cuda().float()
            gt_item = gt_item.cuda().float()
            output_item = model(ft_item)
            val_loss = loss_func(output_item.max(), gt_item.max())
            val_loss += val_loss.item()
    val_loss /= len(val_loader)
    print(f"[Epoch {epoch+1}/{num_epochs}], Validation Loss: {val_loss:.6f}")

print("Done!")


Epoch [1/20], Step [10/579], Loss: 0.568843
Epoch [1/20], Step [20/579], Loss: 0.079337
Epoch [1/20], Step [30/579], Loss: 0.270362
Epoch [1/20], Step [40/579], Loss: 0.507303
Epoch [1/20], Step [50/579], Loss: 0.125846
Epoch [1/20], Step [60/579], Loss: 0.951370
Epoch [1/20], Step [70/579], Loss: 0.645626
Epoch [1/20], Step [80/579], Loss: 0.106065
Epoch [1/20], Step [90/579], Loss: 0.163647
Epoch [1/20], Step [100/579], Loss: 0.309826
Epoch [1/20], Step [110/579], Loss: 0.484784
Epoch [1/20], Step [120/579], Loss: 0.271811
Epoch [1/20], Step [130/579], Loss: 0.165150
Epoch [1/20], Step [140/579], Loss: 0.144420
Epoch [1/20], Step [150/579], Loss: 0.693592
Epoch [1/20], Step [160/579], Loss: 0.193759
Epoch [1/20], Step [170/579], Loss: 0.363684
Epoch [1/20], Step [180/579], Loss: 0.122058
Epoch [1/20], Step [190/579], Loss: 0.324463
Epoch [1/20], Step [200/579], Loss: 0.137201
Epoch [1/20], Step [210/579], Loss: 0.136741
Epoch [1/20], Step [220/579], Loss: 0.207774
Epoch [1/20], Step 

### **第十步**
- 模型推理部分, 通过加载模型使用测试数据作为输入, 得到预测结果
- 其中test_data_path需要给出从下载测试数据解压后的目录路径

In [25]:
# Inference
# Load the model weights
model.load_state_dict(torch.load('./model/model_weights_0.pth'))
model.eval()
import os

test_data_path = "/ai_share/caozhibin/tianchi_precipatition_prediction/test"
os.makedirs("./output", exist_ok=True)
for index, test_data_file in enumerate(os.listdir(test_data_path)):
    test_data = torch.load(os.path.join(test_data_path, test_data_file))
    test_data = test_data.cuda().float()
    
    # Forward pass
    output_item = model(test_data)
    output_item = output_item.to(torch.float16)
    
    # Print the output shape
    print(f"Output shape for sample {test_data_file.split('.')[0]}: {output_item.shape}, {output_item.dtype == torch.float16}")
    print(f"{test_data_file.split('.')[0]}: max: {output_item.max()}, min: {output_item.min()}")
    
    # Save the output
    output_path = f"output/{test_data_file}"
    torch.save(output_item.cpu(), output_path)

Output shape for sample 000: torch.Size([1, 72, 57, 81]), True
000: max: 29.109375, min: 0.0
Output shape for sample 001: torch.Size([1, 72, 57, 81]), True
001: max: 18.59375, min: 0.0
Output shape for sample 002: torch.Size([1, 72, 57, 81]), True
002: max: 30.0, min: 0.0
Output shape for sample 003: torch.Size([1, 72, 57, 81]), True
003: max: 36.96875, min: 0.0
Output shape for sample 004: torch.Size([1, 72, 57, 81]), True
004: max: 15.515625, min: 0.0
Output shape for sample 005: torch.Size([1, 72, 57, 81]), True
005: max: 34.09375, min: 0.0
Output shape for sample 006: torch.Size([1, 72, 57, 81]), True
006: max: 8.125, min: 0.0
Output shape for sample 007: torch.Size([1, 72, 57, 81]), True
007: max: 56.0, min: 0.0
Output shape for sample 008: torch.Size([1, 72, 57, 81]), True
008: max: 9.40625, min: 0.0
Output shape for sample 009: torch.Size([1, 72, 57, 81]), True
009: max: 27.4375, min: 0.0
Output shape for sample 010: torch.Size([1, 72, 57, 81]), True
010: max: 12.890625, min: 0.